In [ ]:
%pip install -r ../requirements.txt

In [ ]:
dbutils.library.restartPython()

In [ ]:
from dbx.pixels.dicom import StowProcessor


In [ ]:
# Parameters passed by the STOW-RS handler via job_parameters in run-now.
catalog_table = dbutils.widgets.get("catalog_table")
volume = dbutils.widgets.get("volume")

assert catalog_table, "catalog_table is required"
assert volume, "volume is required"

_parts = catalog_table.split(".")
assert len(_parts) == 3, "catalog_table must be catalog.schema.table"
stow_table = f"{_parts[0]}.{_parts[1]}.stow_operations"

volume_path = f"/Volumes/{volume.replace('.', '/')}"
checkpoint = f"{volume_path}/_checkpoints/stow_split"

print(f"STOW tracking table : {stow_table}")
print(f"Volume              : {volume}")
print(f"Checkpoint          : {checkpoint}")


In [ ]:
# Phase 1: Split multipart bundles → save individual DICOMs → MERGE status + output_paths
# After this task completes the handler returns extracted paths to the client.

processor = StowProcessor(spark=spark)

processor.split_bundles(
    source_table=stow_table,
    volume=volume,
    checkpoint_location=checkpoint,
)
